In [29]:
import numpy as np
import os

import warnings  # Ignore sklearn future warning
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import argparse
import torch
import random
from lib.fMRI import fMRIAutoencoderDataset, fMRI_Time_Subjs_Embed_Dataset
from lib.helper import extract_hidden_reps, drive_decoding, get_models, checkexist
from torch.utils.data import DataLoader
import os
import datetime

In [30]:
# folder = "/gpfs/milgram/scratch60/turk-browne/tw496/results/forrest_MNI152_3mm_data_early_visual_ind_mrAE_2half/hiddim64_zdim20_lam_mani0.1_bs64"
# al_hidden_path = 'pt5e4000_symmFalse_amlp.npy'
# hidden_path = 'pt5e4000_symmFalse_manihidden.npy'

# labelpath = "/gpfs/milgram/scratch60/turk-browne/neuromanifold/StudyForrest/forrest_movie_labels_coded_expanded.csv"

# ROI = 'early_visual'
# pt = 10
# hidden_dim = 64
# zdim = 20
# lam_mani = 0.1
# batch_size = 64
# TR_range = 2
# symm = False

# TRs = np.arange(1800, 3599)

In [31]:
# al_hidden = np.load(os.path.join(folder, al_hidden_path))
# al_hidden.shape

In [32]:
# hidden = np.load(os.path.join(folder, hidden_path))
# hidden.shape

In [33]:
# np.load("/gpfs/milgram/scratch60/turk-browne/tw496/results/forrest_paper_10pt_hiddendim64_bs64_mlp_md_mse_reg_lam5.0_manilam0.1_phate_shf/data_forrest_paper_10pt_hiddendim64_bs64_mlp_md_mse_reg_lam5.0_manilam0.1_phate_shf_model_on_test_data_e4000_amlp.npy").shape

In [34]:
ROI = "late_visual_neurosynth"

folder = f"/gpfs/milgram/scratch60/turk-browne/tw496/results/forrest_MNI152_3mm_data_{ROI}_ind_mrAE_2half/hiddim64_zdim20_lam_mani0.1_bs64"
labelpath = "/gpfs/milgram/scratch60/turk-browne/neuromanifold/StudyForrest/forrest_movie_labels_coded_expanded.csv"

dfs = []

pt = 10
hidden_dim = 64
zdim = 20
lam_mani = 0.1
batch_size = 64
TR_range = 2
symm = False
# TRs = np.arange(1800, 3599)
# TRs = np.arange(1799)

for train_half in [1,2]:

#     folder = f"/gpfs/milgram/scratch60/turk-browne/tw496/results/forrest_MNI152_3mm_data_early_visual_ind_mrAE_{train_half}half/hiddim64_zdim20_lam_mani0.1_bs64"
    
    if train_half == 1:
        TRs = np.arange(1800, 3599)
    else:
        TRs = np.arange(1799)
        
    for pt in [1, 2, 3, 4, 5, 6, 9, 10, 14, 15]:
        
        print(f'----- Subject: {pt} -- Half: {train_half}-----')

        al_hidden_path = f'pt{pt}e4000_symmFalse_amlp.npy'
        hidden_path = f'pt{pt}e4000_symmFalse_manihidden.npy'

        al_hidden = np.load(os.path.join(folder, al_hidden_path))
        al_hidden.shape

        hidden = np.load(os.path.join(folder, hidden_path))
        hidden.shape

        outdf = None
        cols = ['ROI', 'subj', 'hidden_dim', 'zdim',
                'lam_mani', 'bs',
                'train_half', 'symm_design']
        entry = [ROI, pt, hidden_dim, zdim, lam_mani, batch_size, TR_range, symm]
        # if os.path.exists(SUMMARYFILE):
        #     outdf_old = pd.read_csv(SUMMARYFILE)
        # else:
        #     outdf_old = None
        # if outdf_old is not None:
        #     exist = checkexist(outdf_old, dict(zip(cols, entry)))
        #     if exist:
        #         print(f"{entry} exists")
        #         return

        outdf = pd.DataFrame(columns=cols)

        labeldf = pd.read_csv(labelpath)

        for labeltype in labeldf.columns.tolist()[2:-1]:
            labels = labeldf[labeltype].values[TRs]
            results = drive_decoding(hidden, labels, balance_min=False)
            cols.append(labeltype+'_mani_acc')
            entry.append(results['accuracy'].mean())
            cols.append(labeltype+'_mani_std')
            entry.append(results['accuracy'].std())

        outdf = pd.DataFrame(columns=cols)
        outdf.loc[len(outdf)]=entry
        # outdf.loc[len(outdf)]=entry2

        outdf.loc[len(outdf)-1, 'time' ]=datetime.datetime.now()


        dfs.append(outdf)
        # if os.path.exists(SUMMARYFILE):
        #     outdf_old = pd.read_csv(SUMMARYFILE)
        #     outdf = pd.concat([outdf_old, outdf])

        # resultsdf.loc[len(resultsdf)-1, labeltype+'_acc']=results['accuracy'].mean()
        #                 resultsdf.loc[len(resultsdf)-1, labeltype+'_std']=results['accuracy'].std()

----- Subject: 1 -- Half: 1-----
augmenting training for label = [1.] from count = [503] to count = [936]
augmenting training for label = [1.] from count = [493] to count = [946]
augmenting training for label = [1.] from count = [509] to count = [930]
augmenting training for label = [1.] from count = [499] to count = [940]
augmenting training for label = [1.] from count = [508] to count = [932]
time consumption=  1.484576940536499
augmenting training for label = [-1.] from count = [78] to count = [843]
augmenting training for label = [-1.] from count = [71] to count = [829]
augmenting training for label = [-1.] from count = [71] to count = [857]
augmenting training for label = [-1.] from count = [75] to count = [837]
augmenting training for label = [-1.] from count = [73] to count = [834]
time consumption=  0.42177700996398926
augmenting training for label = [0.] from count = [364] to count = [1075]
augmenting training for label = [0.] from count = [364] to count = [1075]
augmenting tr

augmenting training for label = [-1.] from count = [75] to count = [837]
augmenting training for label = [-1.] from count = [73] to count = [834]
time consumption=  0.4408259391784668
augmenting training for label = [0.] from count = [364] to count = [1075]
augmenting training for label = [0.] from count = [364] to count = [1075]
augmenting training for label = [0.] from count = [377] to count = [1062]
augmenting training for label = [0.] from count = [373] to count = [1066]
augmenting training for label = [0.] from count = [374] to count = [1066]
time consumption=  1.3980412483215332
----- Subject: 10 -- Half: 1-----
augmenting training for label = [1.] from count = [503] to count = [936]
augmenting training for label = [1.] from count = [493] to count = [946]
augmenting training for label = [1.] from count = [509] to count = [930]
augmenting training for label = [1.] from count = [499] to count = [940]
augmenting training for label = [1.] from count = [508] to count = [932]
time cons

augmenting training for label = [1.] from count = [454] to count = [985]
augmenting training for label = [1.] from count = [469] to count = [970]
augmenting training for label = [1.] from count = [464] to count = [975]
augmenting training for label = [1.] from count = [474] to count = [966]
time consumption=  1.4059512615203857
augmenting training for label = [-1.] from count = [53] to count = [889]
augmenting training for label = [-1.] from count = [58] to count = [879]
augmenting training for label = [-1.] from count = [50] to count = [883]
augmenting training for label = [-1.] from count = [45] to count = [879]
augmenting training for label = [-1.] from count = [50] to count = [894]
time consumption=  0.34259486198425293
augmenting training for label = [0.] from count = [290] to count = [1149]
augmenting training for label = [0.] from count = [289] to count = [1150]
augmenting training for label = [0.] from count = [309] to count = [1130]
augmenting training for label = [0.] from co

augmenting training for label = [0.] from count = [290] to count = [1149]
augmenting training for label = [0.] from count = [289] to count = [1150]
augmenting training for label = [0.] from count = [309] to count = [1130]
augmenting training for label = [0.] from count = [303] to count = [1136]
augmenting training for label = [0.] from count = [309] to count = [1131]
time consumption=  1.3003406524658203


In [35]:
resultsdf = pd.concat(dfs)

In [36]:
resultsdf

,ROI,subj,hidden_dim,zdim,lam_mani,bs,train_half,symm_design,IoE_coded_mani_acc,IoE_coded_mani_std,FoT_coded_mani_acc,FoT_coded_mani_std,ToD_coded_mani_acc,ToD_coded_mani_std,time
0,late_visual_neurosynth,1,64,20,0.1,64,2,False,0.593671,0.038561,0.550864,0.034357,0.632043,0.030703,2021-10-05 16:15:05.047100
0,late_visual_neurosynth,2,64,20,0.1,64,2,False,0.573666,0.024048,0.559206,0.031100,0.611998,0.010909,2021-10-05 16:15:08.317351
0,late_visual_neurosynth,3,64,20,0.1,64,2,False,0.585303,0.025591,0.555877,0.023387,0.633686,0.023087,2021-10-05 16:15:11.522113
0,late_visual_neurosynth,4,64,20,0.1,64,2,False,0.564215,0.016178,0.548092,0.026630,0.639803,0.013570,2021-10-05 16:15:14.762197
0,late_visual_neurosynth,5,64,20,0.1,64,2,False,0.566411,0.022067,0.556982,0.030224,0.654802,0.023554,2021-10-05 16:15:18.029802
0,late_visual_neurosynth,6,64,20,0.1,64,2,False,0.573649,0.024483,0.553654,0.030112,0.647591,0.018890,2021-10-05 16:15:21.369198
0,late_visual_neurosynth,9,64,20,0.1,64,2,False,0.585877,0.021953,0.550319,0.029900,0.649248,0.013570,2021-10-05 16:15:24.628175
0,late_visual_neurosynth,10,64,20,0.1,64,2,False,0.564223,0.025304,0.559763,0.023682,0.657038,0.021800,2021-10-05 16:15:27.860184
0,late_visual_neurosynth,14,64,20,0.1,64,2,False,0.571985,0.037764,0.555314,0.032651,0.631456,0.035298,2021-10-05 16:15:31.193279
0,late_visual_neurosynth,15,64,20,0.1,64,2,False,0.605337,0.014119,0.560874,0.017671,0.641469,0.019710,2021-10-05 16:15:34.490447


In [37]:
round(resultsdf['IoE_coded_mani_acc'].mean(), 6)

0.581881

In [38]:
round(resultsdf['IoE_coded_mani_std'].mean(), 6)

0.024637

In [39]:
print(round(resultsdf['FoT_coded_mani_acc'].mean(), 6))
print(round(resultsdf['FoT_coded_mani_std'].mean(), 6))
print('----')
print(round(resultsdf['ToD_coded_mani_acc'].mean(), 6))
print(round(resultsdf['ToD_coded_mani_std'].mean(), 6))

0.577264
0.022704
----
0.664178
0.023183
